In [1]:
import numpy as np
import pandas as pd
import os
import ast
from datetime import datetime
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

import pdb

import requests
from tqdm import tqdm_notebook as tqdm

In [2]:
path = './TweetScraper-master/Data/newSearchTweets'
tweets_df = pd.DataFrame()
toremove = dict.fromkeys((ord(c) for c in u'/xa0/n/t'))

pbar = tqdm(total=(87491))

for file in os.listdir(path):
    if '.DS_Store' not in file:
        pbar.update(1)
        with open(path+'/'+file, 'rb') as f:
            data = pickle.load(f)
            data['datetime'] = datetime.strptime(data['datetime'], '%Y-%m-%d %H:%M:%S')
            data['ID'] = int(data['ID'])
            data['text'] = data['text'].translate(toremove)
            tweets_df = tweets_df.append(data, ignore_index=True)
            
pbar.close()

In [3]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85535 entries, 0 to 85534
Data columns (total 13 columns):
ID               85535 non-null float64
datetime         85535 non-null datetime64[ns]
is_reply         85535 non-null float64
is_retweet       85535 non-null float64
nbr_favorite     85535 non-null float64
nbr_reply        85535 non-null float64
nbr_retweet      85535 non-null float64
text             85535 non-null object
url              85535 non-null object
user_id          85535 non-null object
usernameTweet    85535 non-null object
has_media        29059 non-null float64
medias           29059 non-null object
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 8.5+ MB


In [9]:
tweets_df.head()

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias
0,8.794407e+17,2017-06-26 16:46:13,1.0,0.0,0.0,1.0,0.0,"Lost me on that one, Nick \nThink it's the te...",/PJSanderson/status/879440684624871424,168649848,PJSanderson,NaN,NaN
1,9.830839e+17,2018-04-08 16:47:12,1.0,0.0,3.0,0.0,0.0,"Not only that, but freedom of speech is only p...",/SeamusWalsh2/status/983083861763473408,343861762,SeamusWalsh2,NaN,NaN
2,9.895482e+17,2018-04-26 12:54:16,0.0,0.0,4.0,0.0,1.0,A Fantastic Video From The Archives - Coinsc...,/coinscrum/status/989548224171167744,1196956998,coinscrum,1.0,[https://t.co/ra4K4ln6yx]
3,9.626423e+17,2018-02-11 05:59:37,0.0,0.0,48.0,8.0,7.0,Every morning I read through the posts from ...,/whatbitcoindid/status/962642274596335616,893818632089763840,whatbitcoindid,NaN,NaN
4,9.915655e+17,2018-05-02 02:30:13,0.0,0.0,2.0,0.0,0.0,"""Ask them why they idle there\nWhile we suffer...",/poetrypotion/status/991565507051577344,62152913,poetrypotion,NaN,NaN


In [4]:
tweets_df.to_pickle('./goodUserCrypto.pkl')

In [5]:
tweets_df.sort_values('datetime')

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias
33779,9.845819e+17,2018-04-12 20:00:00,0.0,0.0,0.0,0.0,0.0,"Bicoi (1.2): $7,889.25\nEhereum (1.49): $492.9...",/top_3_coin/status/984581933828005888,926126333951860736,top_3_coin,NaN,NaN
45995,9.845819e+17,2018-04-12 20:00:01,0.0,0.0,0.0,0.0,0.0,Bicoi:$7883.7\nEhereum:$492.457\nBicoi Csh:$73...,/ScrapeINT/status/984581940589277184,904814672444043264,ScrapeINT,NaN,NaN
50632,9.845819e+17,2018-04-12 20:00:01,0.0,0.0,0.0,0.0,0.0,"@ #3, Ripple wih ui price of $.63, mrke cp o...",/CryptoHourly/status/984581937108082688,872234445499244552,CryptoHourly,NaN,NaN
68837,9.845819e+17,2018-04-12 20:00:01,1.0,0.0,0.0,1.0,0.0,You should ge h checked ou. Souds serious if y...,/bechokoy_betts/status/984581937003020288,941782560408875008,bechokoy_betts,NaN,NaN
19628,9.845819e+17,2018-04-12 20:00:02,0.0,0.0,0.0,0.0,0.0,"Mrke Cp: $314,5,284,384\n BTC Domice: 42.61%...",/BotBittrex/status/984581942644527104,921082875839119360,BotBittrex,NaN,NaN
9324,9.845819e+17,2018-04-12 20:00:02,0.0,0.0,0.0,0.0,0.0,"# XRP ‘1 Perce No #Securiy ,’ #Ripple Clims...",/pureinvestments/status/984581941335818240,913434712777117697,pureinvestments,1.0,[https://t.co/nxR28zDjb2]
65414,9.845819e+17,2018-04-12 20:00:02,0.0,0.0,0.0,0.0,0.0,All $ SNGLS Trges Achieved -> hps: goo.glHg...,/ZQIEk1dDPDKGyO3/status/984581944162873346,913731132532936707,ZQIEk1dDPDKGyO3,1.0,[https://t.co/zydx4CFIzn]
69207,9.845819e+17,2018-04-12 20:00:03,0.0,0.0,1.0,1.0,0.0,Top 1 Meios o Twier i he ls hour - $ BTC $ ...,/CoinTrendz/status/984581947312746496,734390196771381248,CoinTrendz,NaN,NaN
42565,9.845820e+17,2018-04-12 20:00:04,0.0,0.0,0.0,0.0,0.0,Our $ PRL brod sigl did 26% i 1 dy. check ...,/w26WcNEE9kRF80B/status/984581951825817600,913731198966550534,w26WcNEE9kRF80B,1.0,[https://t.co/YTmBPkgMw8]
64407,9.845820e+17,2018-04-12 20:00:05,0.0,0.0,0.0,0.0,0.0,1 immese coi which will give you 5% check here...,/lpww0kjEyUWs05M/status/984581955370045440,913732866399510528,lpww0kjEyUWs05M,1.0,[https://t.co/sKyvui0JpX]


In [6]:
subset = tweets_df[tweets_df['datetime']>datetime.strptime('2018-04-13 00:00:00', '%Y-%m-%d %H:%M:%S')]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84260 entries, 0 to 85534
Data columns (total 13 columns):
ID               84260 non-null float64
datetime         84260 non-null datetime64[ns]
is_reply         84260 non-null float64
is_retweet       84260 non-null float64
nbr_favorite     84260 non-null float64
nbr_reply        84260 non-null float64
nbr_retweet      84260 non-null float64
text             84260 non-null object
url              84260 non-null object
user_id          84260 non-null object
usernameTweet    84260 non-null object
has_media        28663 non-null float64
medias           28663 non-null object
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 9.0+ MB


In [7]:
userGroup = subset.groupby('user_id').count()
userGroup = userGroup.sort_values('ID', ascending=False)

In [8]:
userGroup

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,usernameTweet,has_media,medias
user_id,,,,,,,,,,,,
913731198966550534,1212,1212,1212,1212,1212,1212,1212,1212,1212,1212,1205,1205
913732866399510528,1202,1202,1202,1202,1202,1202,1202,1202,1202,1202,1195,1195
913731132532936707,1201,1201,1201,1201,1201,1201,1201,1201,1201,1201,1195,1195
913731301211074563,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1188,1188
948228057063714816,860,860,860,860,860,860,860,860,860,860,860,860
3010205982,758,758,758,758,758,758,758,758,758,758,18,18
2957304731,681,681,681,681,681,681,681,681,681,681,129,129
921082875839119360,576,576,576,576,576,576,576,576,576,576,8,8
909009874498940928,575,575,575,575,575,575,575,575,575,575,0,0


In [21]:
subset[subset['user_id']=='913731198966550534']

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias
4,9.886915e+17,2018-04-24 04:09:56,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 17:09:57\nBTC: 9,999,...",/VirtualMoneyBot/status/988691497930211328,901444148690960384,VirtualMoneyBot,NaN,NaN
13,9.887256e+17,2018-04-24 06:25:31,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 19:25:32\nBTC: 10,070...",/VirtualMoneyBot/status/988725616856911877,901444148690960384,VirtualMoneyBot,NaN,NaN
46,9.886574e+17,2018-04-24 01:54:26,0.0,0.0,1.0,0.0,0.0,"Korea price\nTime: 04/24 14:54:26\nBTC: 10,093...",/VirtualMoneyBot/status/988657396120674304,901444148690960384,VirtualMoneyBot,NaN,NaN
84,9.887029e+17,2018-04-24 04:55:07,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 17:55:08\nBTC: 10,043...",/VirtualMoneyBot/status/988702869288206336,901444148690960384,VirtualMoneyBot,NaN,NaN
100,9.886422e+17,2018-04-24 00:54:11,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 13:54:11\nBTC: 10,049...",/VirtualMoneyBot/status/988642232877043712,901444148690960384,VirtualMoneyBot,NaN,NaN
175,9.886498e+17,2018-04-24 01:24:18,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 14:24:19\nBTC: 10,036...",/VirtualMoneyBot/status/988649815293620224,901444148690960384,VirtualMoneyBot,NaN,NaN
241,9.884786e+17,2018-04-23 14:04:03,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 03:04:03\nBTC: 9,710,...",/VirtualMoneyBot/status/988478625056935936,901444148690960384,VirtualMoneyBot,NaN,NaN
244,9.886077e+17,2018-04-23 22:36:45,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 11:36:46\nBTC: 9,953,...",/VirtualMoneyBot/status/988607650563702784,901444148690960384,VirtualMoneyBot,NaN,NaN
279,9.887597e+17,2018-04-24 08:41:05,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 21:41:06\nBTC: 10,097...",/VirtualMoneyBot/status/988759733904457728,901444148690960384,VirtualMoneyBot,NaN,NaN
311,9.885507e+17,2018-04-23 18:50:34,0.0,0.0,0.0,0.0,0.0,"Korea price\nTime: 04/24 07:50:34\nBTC: 9,732,...",/VirtualMoneyBot/status/988550726669422592,901444148690960384,VirtualMoneyBot,NaN,NaN
